<a href="https://colab.research.google.com/github/KimyYoungWook/DataScience/blob/main/practice/%ED%83%80%EC%8A%88_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
import re
import pandas as pd

In [24]:
data=pd.DataFrame(columns=['스테이션명','위치','상태정보','위도','경도'])

#크롤링할 타슈사이트https://www.tashu.or.kr/userStationAction.do?process=list  (s는 보안접속:로그인해야함)
url_tashu="http://www.tashu.or.kr/userStationAction.do?process=list"

 #페이지 이동 시 추가로 적어줘야 되는 url
url_add = '&page='

 #전체 페이지수 
total_page = 27

#27 page 에 대한 정보를 추출하는 for문
for index in range(1,total_page+1):
  if index == 1 :
    url = url_tashu # 첫번째 페이지
  else :
    url =url_tashu + url_add +str(index) # 페이지 정보 추가

  #url 에서 HTML 추출
  html = urlopen(url)

  # HTML을 파싱하여 데이터로 변환
  bsObject = BeautifulSoup(html, "html.parser", from_encoding= 'utf-8')

  # history_table이라고 하는 class를 가진 table만 추출
  table = bsObject.find_all('table',{'class','history_table'})

  # table내에서 tr 추출
  tr= table[0].find_all('tr')

  # 첫 tr은 contents가 아니라 데이터 정보를  의미하기 때문에 제외
  tr=tr[1:len(tr)]

  #한페이지의 정보를 크롤링
  for index_tr in range(0,len(tr)):
    td = tr[index_tr].find_all('td')

    #장소
    station = td[0].text.split('.')[1]

    #위치 
    location= td[1].text

    #상태
    condition = td[2].text

    #좌표 추출
    geo = td[3].a.attrs['href'].split('(')[1]
    geo = re.sub('[;)]','',geo)
    detail_location = geo.replace("'","")

    #위도 latitude
    lat = detail_location.split(',')[0]
    #경도 longitude
    lon = detail_location.split(',')[1]

    #data에 크롤링 결과 저장
    data = data.append({'스테이션명': station,
                        '위치':location,
                        '상태정보':condition,
                        '위도':lat,
                        '경도':lon},ignore_index=True)
    print('complete of '+ str(index_tr+1) + 'infomation with' + str(index) + ' page')
  #크롤링이 막히는것을 방ㅇ지
  time.sleep(1)
print('-'*80)
print(data)

#크롤링 데이터 저장하기
data.to_csv('TASHU.csv')



complete of 1infomation with1 page
complete of 2infomation with1 page
complete of 3infomation with1 page
complete of 4infomation with1 page
complete of 5infomation with1 page
complete of 6infomation with1 page
complete of 7infomation with1 page
complete of 8infomation with1 page
complete of 9infomation with1 page
complete of 10infomation with1 page
complete of 1infomation with2 page
complete of 2infomation with2 page
complete of 3infomation with2 page
complete of 4infomation with2 page
complete of 5infomation with2 page
complete of 6infomation with2 page
complete of 7infomation with2 page
complete of 8infomation with2 page
complete of 9infomation with2 page
complete of 10infomation with2 page
complete of 1infomation with3 page
complete of 2infomation with3 page
complete of 3infomation with3 page
complete of 4infomation with3 page
complete of 5infomation with3 page
complete of 6infomation with3 page
complete of 7infomation with3 page
complete of 8infomation with3 page
complete of 9infom

In [12]:
import folium

#코랩에서 제공하는 Folium 버전 확인
folium.__version__

'0.8.3'

In [13]:
#지도 그리기
map = folium.Map(location=[36.351,127.386])
map

In [14]:
#지도 확대해서 그리기
map = folium.Map(location=[36.351,127.386],zoom_start=12)
map

지도 스타일(tiles 옵션)

In [15]:
map = folium.Map(location=[36.351,127.386],zoom_start=12,tiles='OpenstreetMap')
map

In [16]:
#OpenStreetMap,Stamen Terrain(featuring hill shading and natural vegetation colors), Stamen Toner(high-contrast B+W (black and white))
map = folium.Map(location=[36.351,127.386],zoom_start=12,tiles='Stamen Terrain')
map

타슈 데이터 가져오기

In [25]:
df = pd.read_csv('TASHU.csv')
detail_location = df.위치
condition = df.상태정보
print(df.loc[2,'경도'])
df.head()

127.388451


,Unnamed: 0,스테이션명,위치,상태정보,위도,경도
0,0,무역전시관입구(택시승강장),유성구 도룡동 3-8,정상,36.374580,127.386131
1,1,대전컨벤션센터,유성구 도룡동 4-19,정상,36.374389,127.392375
2,2,한밭수목원1,서구 만년동 396,정상,36.368995,127.388451
3,3,초원아파트(104동 버스정류장),서구 만년동 401,정상,36.367945,127.379302
4,4,둔산대공원 입구(버스정류장),서구 둔산2동 1521-10,정상,36.364974,127.388856


지도에 Markers를 이용한 타슈 위치 표시하기

In [ ]:
import sys
sys.setrecursionlimit(10**3) 

In [26]:
#대전 중심에 map 열기
map = folium.Map(location=[36.351,127.386],zoom_start=14)
# 타슈의 상태에 따라서 marker를 다른 색으로 표시하기

for index_draw in range(1,len(condition)):
  if condition[index_draw] =='정상':
    #정상인 경우 파란색
    folium.Marker([df.loc[index_draw,'위도'],df.loc[index_draw,'경도']],
                  popup='정상',icon = folium.Icon(color='blue', icon='info-sign')).add_to(map)
  elif condition[index_draw] == 'Network 에러':
    folium.Marker([df.loc[index_draw,'위도'],df.loc[index_draw,'경도']],
                  popup='네트워크 에러',icon = folium.Icon(color='red', icon='info-sign')).add_to(map)
  else :
    folium.Marker([df.loc[index_draw,'위도'],df.loc[index_draw,'경도']],
                  popup='집',icon = folium.Icon(color='green', icon='info-sign')).add_to(map)

In [27]:
map